# Introduction
This notebook will identify similar products based solely on their product names.

To do this the following steps are taken
1. Convert product title to a vector
2. Compute a similarity score and identify similar products
3. Identify the optimal threshold
4. Create submission file


In [1]:
import pandas as pd
import numpy as np
import csv
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Load data
df = pd.read_csv("/kaggle/input/shopee-product-matching/train.csv")
df.head()

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069


# Step 0: Build the actual list of matches

First things first, let's determine what should be the correct output of our submission file. The label_group column contains which group a posting belongs, so postings in the same label_group are deemed as similar.

To do this, we'll group the rows based on their `label_group` and create a list of `posting_id`s that belong to each label_group. Afterwards, we'll add a new column, `actual` to the dataframe containing the posting_ids that belong to the row's label_group

In [3]:
# get list of posting_ids per label group
actual = df.groupby('label_group').posting_id.agg('unique').to_dict()

# add new column, actual, into the dataframe based on row's label_group
df['actual'] = df.label_group.map(actual)

df.head()

,posting_id,image,image_phash,title,label_group,actual
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,"[train_129225211, train_2278313361]"
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,"[train_3386243561, train_3423213080]"
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,"[train_2288590299, train_3803689425]"
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,"[train_2406599165, train_3342059966]"
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,"[train_3369186413, train_921438619]"


While making this notebook, I noticed something interesting with row 3. It should only match postings *train_2406599165* and *train_3342059966* but later on in the notebook we'll see that *train_1744956981* has the exact same title but the label_group is different.

If we look at their image_phash, we can see that they are diifferent meaning they were posted with different images. This might be the reason why they are classified as different groups. It will be good to take note of this when combining the predictions made in this notebook with the predictions of a model that analyzes the images but for now, we'll just ignore it.

In [4]:
# See an example where titles are exactly the same but label group is different
df.loc[df['posting_id'].isin(['train_2406599165', 'train_1744956981'])]

,posting_id,image,image_phash,title,label_group,actual
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,"[train_2406599165, train_3342059966]"
28878,train_1744956981,d81acc9a273166cbfa8c85ca34d31f5f.jpg,abc28c0dd1a4d37e,Daster Batik Lengan pendek - Motif Acak / Camp...,3150867956,"[train_1508100548, train_1744956981]"


Let's export this to a csv file to use later when we compare with our submission file

In [5]:
# convert the actual column into a space delimited string, similar to the output file
#df['actual_str'] = df.apply(lambda row: ' '.join(row['actual']), axis=1)

# export the posting_id and actual_str columns
#df[['posting_id', 'actual_str']].to_csv('actuals.csv', index=False)

# examine the created file
#actuals_df = pd.read_csv('actuals.csv')
#actuals_df.head()

# Step 1: Convert product title to vector
Let's begin! Computers are good with processing numbers but they don't know how to process text. To resolve this, we want to find a way to represent the titles as numbers instead of text. We'll use the **TF-IDF algorithm** to represent the text as a vector. The vector contains a weight for each word in the vocabulary. This will show which words best represent the title.

First, we extract the product titles. Then we use sklearn's TfidfVectorizer to generate the weights per word

In [6]:
# get titles of products
titles = df['title']

# create a tfidfvectorizer model
tfidf_vectorizer = TfidfVectorizer()
tfidf_weights = tfidf_vectorizer.fit_transform(titles)

titles.head()

0                            Paper Bag Victoria Secret
1    Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...
2          Maling TTS Canned Pork Luncheon Meat 397 gr
3    Daster Batik Lengan pendek - Motif Acak / Camp...
4                    Nescafe \xc3\x89clair Latte 220ml
Name: title, dtype: object

Let's examine the weights generated

In [7]:
print("tfidf_weights.shape=", tfidf_weights.shape)

# tfidf_weights is a sparse matrix. Let's convert it to a data frame to take a peak at the data
#df_tfidf_weights = pd.DataFrame(data = tfidf_weights.toarray(), columns=tfidf_vectorizer.get_feature_names())
#df_tfidf_weights.head()

tfidf_weights.shape= (34250, 25069)


We can see that from the 34,250 products there are 25,069 unique words used in the titles

# Step 2: Compute similarity score and identify similar products

Next we need to compute how similar the products are to each other. Since there are 34,250 products, we'll be generating a 34,250 x 34,250 matrix.

The score can be computed by using either Euclidean distance or cosine similarity. *(This [article](https://www.baeldung.com/cs/euclidean-distance-vs-cosine-similarity) explains the difference really well)*
- Euclidean distance, aka L2-norm, measures the distance between 2 points in a vector space
- Cosine similarity, on the otherhand, does not treat the data as points rather as vectors from the origin the measure. This way alows us to compute the anfular distance, which basically means how close the vectors are to each other.

We'll be computing both scores then see later which one performs better. Also for now, let's only say that products are the same only if their titles are exactly the same

Note: To avoid any issues regarding space allocation for the 70,000 records in the hidden test set, we'll be copmputing the distances in chunks

In [8]:
# set the chuck size
CHUNK_SIZE = 1024*4
print("CHUNK_SIZE=", CHUNK_SIZE)

# compute how many chunks we need to process
CHUNKS = df.shape[0]//CHUNK_SIZE if df.shape[0] % CHUNK_SIZE == 0 else df.shape[0]//CHUNK_SIZE + 1
print("CHUNKS=", CHUNKS)

CHUNK_SIZE= 4096
CHUNKS= 9


## Euclidean Distance

Below is the code to compute for the euclidean distance between 2 product titles

In [28]:
from sklearn.metrics.pairwise import euclidean_distances

def predict_by_euclidean(threshold, verbose=True):
    print("############ Predict using euclidean distance")
    print("############ THRESHOLD = ", threshold)
    
    # initialize the list where we'll store the predictions
    predictions = []
    maxs = []

    # compute the euclidean distance per chunk
    for i in range(CHUNKS):
        # compute the start and end indexes of the chunk
        start_index = i * CHUNK_SIZE
        end_index = start_index + CHUNK_SIZE
        if(end_index > df.shape[0]):
            end_index = df.shape[0]
        
        if verbose:
            print("Processing records", start_index, "to", end_index)

        e_distance = euclidean_distances(tfidf_weights[start_index:end_index], tfidf_weights)

        # loop through each record
        for j in range(0,e_distance.shape[0]):
            # get the indices of products with exactly the same title
            indices = np.where(e_distance[j,:] <= threshold)

            # get posting ids
            posting_ids = df.loc[indices,'posting_id'].to_list()

            # add these indices to a list
            predictions.append(posting_ids)
        
        maxs.append(e_distance.max())
    
    if verbose:
        print("Largest difference=", e_distance.max())

    return predictions

The code below will call the function above and add our predictions to the dataframe as a new column, e_prediction.

In [12]:
df['e_prediction'] = predict_by_euclidean(0.9)
df.head()

Processing records 0 to 4096
Processing records 4096 to 8192
Processing records 8192 to 12288
Processing records 12288 to 16384
Processing records 16384 to 20480
Processing records 20480 to 24576
Processing records 24576 to 28672
Processing records 28672 to 32768
Processing records 32768 to 34250
Largest difference= 1.4142135623730956


,posting_id,image,image_phash,title,label_group,actual,e_prediction
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,"[train_129225211, train_2278313361]","[train_129225211, train_2278313361]"
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,"[train_3386243561, train_3423213080]","[train_3386243561, train_860027362, train_3423..."
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,"[train_2288590299, train_3803689425]","[train_2288590299, train_3803689425]"
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,"[train_2406599165, train_3342059966]","[train_2406599165, train_3576714541, train_150..."
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,"[train_3369186413, train_921438619]",[train_3369186413]


## Cosine Similarity

To compute the cosine similarity we just simple need to take the dot product between 2 postings

In [27]:
import scipy.sparse as sp

def predict_by_cosine_similarity(threshold, verbose=True):
    print("############ Predict using cosine similarity")
    print("############ THRESHOLD = ", threshold)
    
    # initialize the list where we'll store the predictions
    predictions = []

    # compute the cosine similarity distance per chunk
    for i in range(CHUNKS):
        # compute the start and end indexes of the chunk
        start_index = i * CHUNK_SIZE
        end_index = start_index + CHUNK_SIZE
        if(end_index > df.shape[0]):
            end_index = df.shape[0]
        
        if verbose:
            print("Processing records", start_index, "to", end_index)

        # compute cosine similarity distance
        cs_distance = tfidf_weights[start_index:end_index].dot(tfidf_weights.T)
        #print("cs_distance.shape = ",cs_distance.shape)

        # get posting_ids where cosine similarity distance >= threshold
        for j in range(0,cs_distance.shape[0]):
            # get values that are <= threshold
            valid_distances = cs_distance[j,:] >= threshold
            #print("valid_distances.shape = ",valid_distances.shape)
            
            # get indices from cs_distance matrix
            row_idx, col_idx, _ = sp.find(valid_distances)

            # get corresponding posting_id of indices
            posting_ids = df.loc[col_idx,'posting_id'].to_list()

            # add posting_ids to predictions list
            predictions.append(posting_ids)
    
    return predictions

In [15]:
df['cs_prediction'] = predict_by_cosine_similarity(0.7)
df.head()

Processing records 0 to 4096
Processing records 4096 to 8192
Processing records 8192 to 12288
Processing records 12288 to 16384
Processing records 16384 to 20480
Processing records 20480 to 24576
Processing records 24576 to 28672
Processing records 28672 to 32768
Processing records 32768 to 34250


,posting_id,image,image_phash,title,label_group,actual,e_prediction,cs_prediction
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,"[train_129225211, train_2278313361]","[train_129225211, train_2278313361]","[train_129225211, train_2278313361]"
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,"[train_3386243561, train_3423213080]","[train_3386243561, train_860027362, train_3423...",[train_3386243561]
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,"[train_2288590299, train_3803689425]","[train_2288590299, train_3803689425]",[train_2288590299]
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,"[train_2406599165, train_3342059966]","[train_2406599165, train_3576714541, train_150...","[train_2406599165, train_3576714541, train_150..."
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,"[train_3369186413, train_921438619]",[train_3369186413],[train_3369186413]


# Step 3: Identifying the optimal threshold

The `predict_by_euclidean` and `predict_by_cosine_similarity` functions that we created above determine that products are similar based on a threshold. There are different possible values that we can use as a threshold so let's identify which one gives us the best accuracy
- **Euclidean Distance** - Based on our data, the smallest distance we can compute is 0 and largest is around 1.41. Therefore, we should look for thresholds between 0 to 1.41. Note that titles are exactly the same if the euclidean distance is 0
- **Cosine Similarity** - The values we can compute from cosine similarity ranges from 0 to 1. Also note that titles are exactly the same if we get a value of 1

For this competition, accuracy is determined based on the F1 score and can be computed as follows: (Formula was referenced from this [notebook](https://www.kaggle.com/c/shopee-product-matching/discussion/225093))

$$
F1 Metric = \frac{2 * \mid X \cap Y \mid}{\mid X\mid + \mid Y\mid}
$$

In [16]:
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.actual,row[col]) )
        return 2*n / (len(row.actual)+len(row[col]))
    return f1score

Let's see what's the optimal threshold if we use euclidean distance

In [17]:
# set threshold size
THRESHOLD_VALUES = [0, 0.3, 0.6, 0.9, 1, 1.3] # Note: The largest distance is around 1.41 so going beyond 1.3 is no longer productive
cv_scores = []

for threshold in THRESHOLD_VALUES:
    # get predictions using threshold
    predictions = predict_by_euclidean(threshold)
    
    # update data frame with predictions
    df['e_prediction'] = predictions
    
    # compute f1 score per row
    df['f1_e'] = df.apply(getMetric('e_prediction'),axis=1)
    
    # get cv score
    cv_score = df.f1_e.mean()
    cv_scores.append(cv_score)
    print('cv_score =', cv_score)

e_method_accuracies = pd.DataFrame({'threshold':THRESHOLD_VALUES, 'accuracy':cv_scores})
e_method_accuracies.sort_values('accuracy', ascending=False)


############ THRESHOLD =  0
Processing records 0 to 4096
Processing records 4096 to 8192
Processing records 8192 to 12288
Processing records 12288 to 16384
Processing records 16384 to 20480
Processing records 20480 to 24576
Processing records 24576 to 28672
Processing records 28672 to 32768
Processing records 32768 to 34250
Largest difference= 1.4142135623730956
cv_score = 0.49025587795310027
############ THRESHOLD =  0.3
Processing records 0 to 4096
Processing records 4096 to 8192
Processing records 8192 to 12288
Processing records 12288 to 16384
Processing records 16384 to 20480
Processing records 20480 to 24576
Processing records 24576 to 28672
Processing records 28672 to 32768
Processing records 32768 to 34250
Largest difference= 1.4142135623730956
cv_score = 0.49845215989690755
############ THRESHOLD =  0.6
Processing records 0 to 4096
Processing records 4096 to 8192
Processing records 8192 to 12288
Processing records 12288 to 16384
Processing records 16384 to 20480
Processing rec

,threshold,accuracy
3,0.9,0.645999
4,1.0,0.637634
2,0.6,0.560984
1,0.3,0.498452
0,0.0,0.490256
5,1.3,0.124354


Now, let's see what's the optimal threshold for cosine similarity

In [18]:
THRESHOLD_VALUES = [1, 0.9, 0.95, 0.8, 0.85, 0.7, 0.75, 0.6, 0.65, 0.5, 0.55, 0.4]
cv_scores = []

for threshold in THRESHOLD_VALUES:
    print("############ THRESHOLD = ", threshold)
    # get predictions using threshold
    predictions = predict_by_cosine_similarity(threshold)
    
    # update data frame with predictions
    df['cs_prediction'] = predictions
    
    # compute f1 score per row
    df['f1_cs'] = df.apply(getMetric('cs_prediction'),axis=1)
    
    # get cv score
    cv_score = df.f1_cs.mean()
    cv_scores.append(cv_score)
    print('cv_score =', cv_score)

cs_method_accuracies = pd.DataFrame({'threshold':THRESHOLD_VALUES, 'accuracy':cv_scores})
cs_method_accuracies.sort_values('accuracy', ascending=False)

############ THRESHOLD =  1
Processing records 0 to 4096
Processing records 4096 to 8192
Processing records 8192 to 12288
Processing records 12288 to 16384
Processing records 16384 to 20480
Processing records 20480 to 24576
Processing records 24576 to 28672
Processing records 28672 to 32768
Processing records 32768 to 34250
cv_score = 0.3201769568160777
############ THRESHOLD =  0.9
Processing records 0 to 4096
Processing records 4096 to 8192
Processing records 8192 to 12288
Processing records 12288 to 16384
Processing records 16384 to 20480
Processing records 20480 to 24576
Processing records 24576 to 28672
Processing records 28672 to 32768
Processing records 32768 to 34250
cv_score = 0.5211588666283922
############ THRESHOLD =  0.95
Processing records 0 to 4096
Processing records 4096 to 8192
Processing records 8192 to 12288
Processing records 12288 to 16384
Processing records 16384 to 20480
Processing records 20480 to 24576
Processing records 24576 to 28672
Processing records 28672 

,threshold,accuracy
10,0.55,0.648687
7,0.60,0.645288
9,0.50,0.637634
8,0.65,0.634458
5,0.70,0.617276
6,0.75,0.595241
3,0.80,0.570926
11,0.40,0.566242
4,0.85,0.545953
1,0.90,0.521159


Comparing the euclidean and cosine similarity methods, it looks like cosine similarity has the best accuracy so for now let's try to create our submission file using this method and threshold.

In [24]:
print("Best accuracy from cosine method = ", cs_method_accuracies['accuracy'].max())
print("Best accuracy from euclidean method =", e_method_accuracies['accuracy'].max())

if cs_method_accuracies['accuracy'].max() > e_method_accuracies['accuracy'].max():
    prediction_col = 'cs_prediction'
    threshold = cs_method_accuracies.loc[cs_method_accuracies.accuracy == cs_method_accuracies['accuracy'].max(), 'threshold'].item()
else:
    prediction_col = 'e_prediction'
    threshold = e_method_accuracies.loc[e_method_accuracies.accuracy == e_method_accuracies['accuracy'].max(), 'threshold'].item()
    
print("Use", prediction_col, " column with ", threshold, " threshold for submission file")
    
    

Best accuracy from cosine method =  0.6486870337299409
Best accuracy from euclidean method = 0.6459987805022945
Use cs_prediction  column with  0.55  threshold for submission file


# Step 4: Create the submission file

To create the submission file, let's generate the predictions using the ideal threshold

In [29]:
if prediction_col == 'e_prediction':
    predictions = predict_by_euclidean(threshold)
else:
    predictions = predict_by_cosine_similarity(threshold)
    
# update data frame with predictions
df['predictions_final'] = predictions

############ Predict using cosine similarity
############ THRESHOLD =  0.55
Processing records 0 to 4096
Processing records 4096 to 8192
Processing records 8192 to 12288
Processing records 12288 to 16384
Processing records 16384 to 20480
Processing records 20480 to 24576
Processing records 24576 to 28672
Processing records 28672 to 32768
Processing records 32768 to 34250


In [30]:
df['matches'] = df.apply(lambda row: ' '.join(row['predictions_final']), axis=1)
df.head()

,posting_id,image,image_phash,title,label_group,actual,e_prediction,cs_prediction,f1_e,f1_cs,predictions_final,matches
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,"[train_129225211, train_2278313361]","[train_129225211, train_655833377, train_34979...","[train_129225211, train_2278313361]",0.050633,1.000000,"[train_129225211, train_2278313361]",train_129225211 train_2278313361
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,"[train_3386243561, train_3423213080]","[train_3386243561, train_1816968361, train_366...","[train_3386243561, train_1816968361, train_212...",0.028369,0.200000,"[train_3386243561, train_860027362, train_7788...",train_3386243561 train_860027362 train_7788161...
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,"[train_2288590299, train_3803689425]","[train_2288590299, train_597673068, train_3135...","[train_2288590299, train_3803689425]",0.444444,1.000000,"[train_2288590299, train_3803689425]",train_2288590299 train_3803689425
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,"[train_2406599165, train_3342059966]","[train_2406599165, train_2088327894, train_975...","[train_2406599165, train_3576714541, train_270...",0.019231,0.181818,"[train_2406599165, train_3576714541, train_150...",train_2406599165 train_3576714541 train_150810...
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,"[train_3369186413, train_921438619]","[train_3369186413, train_2223127215, train_105...",[train_3369186413],0.100000,0.666667,[train_3369186413],train_3369186413


In [31]:
df[['posting_id', 'matches']].to_csv('submission.csv', index=False)
submission_df = pd.read_csv('submission.csv')
submission_df.head()

,posting_id,matches
0,train_129225211,train_129225211 train_2278313361
1,train_3386243561,train_3386243561 train_860027362 train_7788161...
2,train_2288590299,train_2288590299 train_3803689425
3,train_2406599165,train_2406599165 train_3576714541 train_150810...
4,train_3369186413,train_3369186413


# Conclusion

And we're done! We can now predict which postings are the same based on their titles. This approach would give a public score of 0.588.

The next steps would be to try to predict using the `image_phash` and the actual image. We can also combine the results of the 3 methods and see if accuracy will improve